## Clasificación binaria: 3 neuronas con varias columnas en la X
 
La arquitectura que usaremos será:

- **Capa oculta** de **2 neuronas**, cada una con activación ReLU.  
  - Neurona 1: $z_1 = \mathbf{w_1}^\top \mathbf{x} + b_1,\; a_1 = \mathrm{ReLU}(z_1)$.  
  - Neurona 2: $z_2 = \mathbf{w_2}^\top \mathbf{x} + b_2,\; a_2 = \mathrm{ReLU}(z_2)$.  
  - $\mathbf{x}$ tiene dimensión 2 (colesterol, glucosa). Por lo tanto, $\mathbf{w_1}$ y $\mathbf{w_2}$ son cada uno un vector de longitud 2.

- **Capa de salida** (1 neurona) con activación sigmoide para predecir la **probabilidad** de clase 1 (apto).  
  - Entrada: $(a_1, a_2)$.  
  - Parámetros: $\mathbf{w_3}\in \mathbb{R}^2$ y $b_3\in \mathbb{R}$.  
  - $z_3 = w_{3,1} a_1 + w_{3,2} a_2 + b_3$.  
  - $y_{\text{pred}} = \sigma(z_3)$.  

La salida será un escalar entre 0 y 1, interpretado como la **probabilidad** de $y=1$. Usaremos **entropía cruzada binaria** (BCE) como función de costo.

## 1. Dataset

Imaginemos un dataset (ficticio) con dos columnas: `[colesterol, glucosa]`, y la etiqueta binaria `[0,1]` según si es “no apto” o “apto”.

In [14]:
import numpy as np

# X de forma (N, 2)
# columnas: [colesterol, glucosa]
X = np.array([
    [180, 85],
    [200, 90],
    [220, 120],
    [250, 130],
    [270, 150],
    [300, 180]
], dtype=float)

y = np.array([0, 0, 0, 1, 1, 1], dtype=float)

N = X.shape[0]

## Forward pass

Calcula la **salida de cada neurona** en orden.  
1. **Neurona 1**: $z_1, a_1$  
2. **Neurona 2**: $z_2, a_2$  
3. **Neurona de salida**: $z_3, y_{\text{pred}}$

In [15]:
def relu(z):
    return np.maximum(0, z)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def forward(X, w1, b1, w2, b2, w3, b3):
    """
    X: (N,2)
    w1, w2: cada uno (2,)
    b1, b2: escalares
    w3: (2,)  (entradas: a1, a2)
    b3: escalar

    Returns:
      y_pred: shape (N,)  -- prob de ser clase 1
      z1, a1, z2, a2, z3  (intermedios) para backprop
    """
    # Neurona 1
    z1 = np.dot(X, w1) + b1  # (N,)
    a1 = relu(z1)            # (N,)

    # Neurona 2
    z2 = np.dot(X, w2) + b2  # (N,)
    a2 = relu(z2)            # (N,)

    # Neurona de salida
    # Entrada = [a1, a2],   w3 = [w3[0], w3[1]]
    # z3 = w3[0]*a1 + w3[1]*a2 + b3
    # vectorizado:
    z3 = w3[0]*a1 + w3[1]*a2 + b3  # (N,)

    y_pred = sigmoid(z3)          # (N,)
    return y_pred, z1, a1, z2, a2, z3

## Función coste BCE

In [16]:
def binary_cross_entropy(y_pred, y_true):
    epsilon = 1e-7
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true)*np.log(1 - y_pred))

## Gradientes

Aplicaremos la **regla de la cadena** para derivar la BCE (binary cross entropy) respecto a cada parámetro: $w_1, b_1, w_2, b_2, w_3, b_3$.

#### Derivadas para la neurona de salida

Sea $z_3^{(i)} = w_3[0]\cdot a_1^{(i)} + w_3[1]\cdot a_2^{(i)} + b_3$, y $\hat{y}^{(i)} = \sigma(z_3^{(i)})$. La BCE:

$$
\text{BCE} = -\frac{1}{N}\sum_{i=1}^N \bigl[y^{(i)}\log(\hat{y}^{(i)}) + (1-y^{(i)})\log(1-\hat{y}^{(i)})\bigr].
$$

La derivada $\partial \text{BCE}/\partial z_3^{(i)}$ = $\hat{y}^{(i)} - y^{(i)}$ (resultado estándar de la entropía cruzada con sigmoide).

Por ende:

1. $\frac{\partial}{\partial w_3[0]} \text{BCE}$ = $\frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}) \cdot a_1^{(i)}$.  
2. $\frac{\partial}{\partial w_3[1]} \text{BCE}$ = $\frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}) \cdot a_2^{(i)}$.  
3. $\frac{\partial}{\partial b_3} \text{BCE}$ = $\frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)})$.

#### Derivadas para neuronas 1 y 2 (ocultas, con ReLU)

Sea `d3` = $\frac{\partial \text{BCE}}{\partial z_3^{(i)}} = (\hat{y}^{(i)} - y^{(i)})$.  

Para la **Neurona 1** (con $z_1^{(i)} = w_1 \cdot x^{(i)} + b_1$, $a_1^{(i)} = \mathrm{ReLU}(z_1^{(i)})$):

$$
z_3^{(i)} \text{ depende de } a_1^{(i)}, \quad
a_1^{(i)} = \mathrm{ReLU}(z_1^{(i)})
$$
$$
\frac{\partial \text{BCE}}{\partial w_1} 
= \sum_{i=1}^N \bigl[\frac{\partial \text{BCE}}{\partial z_3^{(i)}} \cdot \frac{\partial z_3^{(i)}}{\partial a_1^{(i)}} \cdot \frac{\partial a_1^{(i)}}{\partial z_1^{(i)}} \cdot \frac{\partial z_1^{(i)}}{\partial w_1}\bigr].
$$

- $\frac{\partial z_3^{(i)}}{\partial a_1^{(i)}} = w_3[0]$.  
- $\frac{\partial a_1^{(i)}}{\partial z_1^{(i)}} = \mathbf{1}_{z_1^{(i)} > 0}$ (derivada de ReLU: 1 si $z_1>0$, 0 si no).  
- $\frac{\partial z_1^{(i)}}{\partial w_1} = x^{(i)}$ (vector).  

Por lo tanto, para cada ejemplo $i$:

$$
\text{con } d3^{(i)} = (\hat{y}^{(i)} - y^{(i)}),
\quad
\text{ReLU'}(z_1^{(i)}) = \mathbf{1}_{z_1^{(i)} > 0},
$$
$$
\frac{\partial \text{BCE}}{\partial w_1} 
= \frac{1}{N}\sum_{i=1}^N \Bigl[d3^{(i)} \cdot w_3[0] \cdot \mathbf{1}_{z_1^{(i)} > 0} \cdot x^{(i)}\Bigr].
$$

Análogamente, $\frac{\partial \text{BCE}}{\partial b_1}$ = $\frac{1}{N}\sum d3^{(i)} \cdot w_3[0] \cdot \mathbf{1}_{z_1^{(i)}>0}$.

Lo mismo para la **Neurona 2**, usando $w_3[1]$ y $\mathbf{1}_{z_2^{(i)}>0}$.

In [17]:
def backward(X, y, 
             w1, b1, w2, b2, w3, b3):
    """
    Calcula gradientes de BCE wrt w1,b1, w2,b2, w3,b3.
    Devuelve dw1, db1, dw2, db2, dw3, db3 (todos en forma NumPy).
    """
    N = len(X)
    # 1. Forward para obtener valores intermedios
    y_pred, z1, a1, z2, a2, z3 = forward(X, w1, b1, w2, b2, w3, b3)
    
    # 2. d3 = (y_pred - y), shape (N,)   (para cada ejemplo)
    d3 = (y_pred - y)  # BCE con sigmoide -> deriv w.r.t z3
    
    # ================================
    # Gradientes wrt w3, b3 (neurona de salida)
    # w3 = [w3[0], w3[1]]
    dw3_0 = np.mean(d3 * a1)  # partial wrt w3[0]
    dw3_1 = np.mean(d3 * a2)  # partial wrt w3[1]
    db3   = np.mean(d3)       # partial wrt b3
    
    dw3 = np.array([dw3_0, dw3_1])  # shape(2,)
    
    # ================================
    # Gradientes wrt neurona 1 (z1 -> a1)
    # d(z3)/da1 = w3[0]
    # d(a1)/dz1 = ReLU'(z1) = 1_{z1>0}
    relu_mask1 = (z1 > 0).astype(float)  # shape(N,)
    da1_dz1 = relu_mask1
    # Se multiplica todo: d3 * w3[0] * da1_dz1
    dz1 = d3 * w3[0] * da1_dz1  # shape(N,)
    # dw1 = mean( X.T * dz1 )
    dw1 = np.dot(X.T, dz1) / N  # shape(2,)
    # db1 = mean(dz1)
    db1 = np.mean(dz1)
    
    # ================================
    # Gradientes wrt neurona 2 (z2 -> a2)
    relu_mask2 = (z2 > 0).astype(float)
    dz2 = d3 * w3[1] * relu_mask2  # shape(N,)
    dw2 = np.dot(X.T, dz2) / N     # shape(2,)
    db2 = np.mean(dz2)
    
    return dw1, db1, dw2, db2, dw3, db3

## Entrenamiento

Ahora tenemos 6 **parámetros** en total:

1. **Neurona 1 (capa oculta):**  
   - $\mathbf{w}_1$ (2,)  
   - $b_1$ (escalar)
2. **Neurona 2 (capa oculta):**  
   - $\mathbf{w}_2$ (2,)  
   - $b_2$ (escalar)
3. **Neurona de salida:**  
   - $\mathbf{w}_3$ (2,)  — pues entra $(a_1, a_2)$  
   - $b_3$ (escalar)

In [18]:
np.random.seed(42)

w1 = np.random.randn(2) * 0.01
b1 = 0.0

w2 = np.random.randn(2) * 0.01
b2 = 0.0

w3 = np.random.randn(2) * 0.01
b3 = 0.0

learning_rate = 0.0001
num_epochs = 5000

for epoch in range(num_epochs):
    # Forward
    y_pred, z1, a1, z2, a2, z3 = forward(X, w1, b1, w2, b2, w3, b3)
    loss_value = binary_cross_entropy(y_pred, y)
    
    # Backward
    dw1, db1, dw2, db2, dw3, db3 = backward(X, y, w1, b1, w2, b2, w3, b3)
    
    # Actualizar parámetros
    w1 -= learning_rate * dw1
    b1 -= learning_rate * db1
    w2 -= learning_rate * dw2
    b2 -= learning_rate * db2
    w3 -= learning_rate * dw3
    b3 -= learning_rate * db3
    
    # (Opcional) imprimir cada cierto número de iteraciones
    if (epoch+1) % 1000 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} | Loss={loss_value:.6f}")

Epoch 1000/5000 | Loss=0.678275
Epoch 2000/5000 | Loss=0.674815
Epoch 3000/5000 | Loss=0.671053
Epoch 4000/5000 | Loss=0.664765
Epoch 5000/5000 | Loss=0.653674


In [ ]:
print("\nEntrenamiento terminado.")
print("w1 =", w1, "b1 =", b1)
print("w2 =", w2, "b2 =", b2)
print("w3 =", w3, "b3 =", b3)

y_pred_final, _, _, _, _, _ = forward(X, w1, b1, w2, b2, w3, b3)
print("Predicciones (prob):", y_pred_final)
print("Clases predichas:   ", (y_pred_final >= 0.5).astype(int))
print("Clases reales:      ", y)


Entrenamiento terminado.
w1 = [ 0.00585881 -0.00298174] b1 = 4.475841610629525e-05
w2 = [-0.02455966  0.08116455] b2 = -0.0017523557559649093
w3 = [-0.00470483  0.08322033] b3 = -0.03526826041500982
Predicciones (prob): [0.54166742 0.53978358 0.5795923  0.5809423  0.60362339 0.63679952]
Clases predichas:    [1 1 1 1 1 1]
Clases reales:       [0. 0. 0. 1. 1. 1.]


In [20]:
def accuracy_score(y_true, y_pred):
    y_pred_classes = (y_pred >= 0.5).astype(int)
    correct_predictions = np.sum(y_true == y_pred_classes)
    accuracy = correct_predictions / len(y_true)
    return accuracy

accuracy_value = accuracy_score(y, (y_pred_final >= 0.5).astype(int))
print(f"Accuracy: {accuracy_value:.2%}")

Accuracy: 50.00%
